In [1]:
from datetime import date
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import date

chandlierMultiplier = 2
max_holding = 100
source = "CSV"

#dateTimeFormat = '%Y-%m-%d'
dateTimeFormat = '%d-%b-%Y'

# filter
days = 30 * 24

# frequency
freq = "1D"
symbol = 'RIIL'
myDate="31_Mar_2019"
filename ="./data/"+symbol+"_"+myDate+".csv"

# Calculate indicators and fields
%run "freedom_indicators_calculation.ipynb"


########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0
    
class Trend(Enum):
    SIDE = 0
    UP = 1
    SUP = 2
    DOWN = 3
    SDOWN = 4

In [32]:
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC

# Math
price['priceSD'] = STDDEV(price.close)
price['priceVAR'] = VAR(price.close)

price['delBBUP'] = (price.bbup - price.close)/(price.bbup-price.bbmid)
price['delBBLOW'] = (price.close - price.bblow)/(price.bbmid-price.bblow)

price['d1Slope'] = np.arctan(price.close - price.close.shift(1)) * 180 / np.pi
price['d3Slope'] = np.arctan(price.close - price.close.shift(3)) * 180 / np.pi
price['d5Slope'] = np.arctan(price.close - price.close.shift(5)) * 180 / np.pi
price['d10Slope'] = np.arctan(price.bbmid - price.bbmid.shift(3)) * 180 / np.pi
price['d15Slope'] = np.arctan(price.close - price.close.shift(15)) * 180 / np.pi

price['macd1dSlope'] = np.arctan(price.macd - price.macd.shift(1)) * 180 / np.pi

price['macdTSF'] = TSF(price.macd)
price['priceTSF'] = TSF(price.close)


price['ROC'] = ROC(price.close)
price['macdROC'] = ROC(price.macd)

TrendThreshold = 85
trend = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
trend[price.d10Slope >= TrendThreshold] = Trend.UP
trend[price.d10Slope <= -1 * TrendThreshold] = Trend.DOWN
#price.trend1D[price1dGradient >= 65] = Trend.SUP
#price.trend1D[price1dGradient <= -65] = Trend.SDOWN

price['trend1D'] = trend

price['sdHighLow'] = STDDEV((price.high - price.close))
price['varHighLow'] = STDDEV((price.high - price.close))
price['delPriceHighLow'] = (price.high - price.close)/price.sdHighLow

price['PriceChange%'] = (price.close - price.shift(1).close)*100/price.close

price.dropna(inplace=True)
labels = pd.DataFrame(data = {"holdings":np.full(price.index.shape[0], S.HOLD)},index=price.index)
    
#labels = pd.DataFrame()

price.columns

counter = 0

for index,row in price[1:].iterrows():
    counter = counter + 1
    prevrow = price.iloc[counter-1]
    if row['ADX'] <= 25: # Market is moving sideways
        if (row.bbup - row.high)/(row.bbup-row.bbmid) <= 0.1:
            labels.iloc[counter] = S.SELL
        elif (row.low - row.bblow)/(row.bbmid-row.bblow) <= 0.1:
            labels.iloc[counter] = S.BUY
    

In [33]:
#Entry Criterion
'''labels[
    ((price.ADX>=25)&(price.macd.shift(1) < price.macdsignal.shift(1)) & (price.macd.shift(3) >= price.macd.shift(2)) & (price.macd>price.macd.shift(1))) |
    ((price.ADX>=25)&(price.macd.shift(3) <= price.macdsignal.shift(3)) & (price.macdhist > 0))
] = S.BUY

#Exit Criterion
labels[
     ((price.ADX <30) & (price.delBBUP<=0.2) )|
     ((price.macd.shift(1) >= price.macdsignal.shift(1)) & (price.macdhist <= 0))
] = S.SELL'''


# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

#################################
#   Plotting Data              #
###############################

rangeMin = price.index[-150]
rangeMax = price.index[-1]

%run "freedom_plot.ipynb"

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]
[ (9,1) x1,y9 ]



## Entry Criterion
- MACD crossover when market is trending up
- BB when market is moving sideways

## Exit Criterion
- Stochastics

## Identify market trend
- low ATR means market moving sideways
- Flat(gradient=0) MACD means market is moving sideways
- low ATR indicates low volatility - market is moving sideways
- High ATR indicates high volatility - market is trending
- Gradient of current - last 15 days


ADX Value	Trend Strength
- 0-25	Absent or Weak Trend
- 25-50	Strong Trend
- 50-75	Very Strong Trend
- 75-100	Extremely Strong Trend

In [49]:
mydf = pd.DataFrame()

In [47]:
mydf.loc[0] = 10

In [63]:
price[1:]

,open,high,low,close,adjClose,volume,macd,macdsignal,macdhist,RSI,...,macd1dSlope,macdTSF,priceTSF,ROC,trend1D,sdHighLow,varHighLow,delPriceHighLow,PriceChange%,macdROC
2018-02-20,482.90,486.50,475.00,476.85,476.85,344361,-22.418609,-21.868893,-0.549716,36.525932,...,4.058203,-28.447794,473.891209,-1.222165,Trend.DOWN,3.617098,3.617098,2.667884,-0.849324,22.492113
2018-02-21,480.00,482.00,475.10,477.90,477.90,205677,-22.023780,-21.899871,-0.123909,36.966836,...,21.545554,-26.366155,479.310989,4.516129,Trend.DOWN,5.873278,5.873278,0.698077,0.219711,-5.296834
2018-02-22,472.00,477.60,467.00,470.70,470.70,232477,-22.037816,-21.927460,-0.110357,35.163140,...,-0.804186,-24.152326,482.851648,2.292731,Trend.DOWN,4.948475,4.948475,1.394369,-1.529637,-17.276029
2018-02-23,472.05,484.00,472.00,479.55,479.55,318872,-21.091688,-21.760305,0.668618,39.096713,...,43.414388,-22.038726,484.153846,0.073038,Trend.DOWN,4.549352,4.549352,0.978161,1.845480,-23.215498
2018-02-26,483.90,491.30,481.10,485.15,485.15,385077,-19.663334,-21.340911,1.677577,41.514633,...,55.003878,-19.995391,485.861538,1.178311,Trend.DOWN,1.992737,1.992737,3.086208,1.154282,-29.219749
2018-02-27,487.00,489.80,477.15,480.35,480.35,260027,-18.703076,-20.813344,2.110268,40.047028,...,43.838557,-18.503767,481.635165,-3.008582,Trend.SIDE,1.924942,1.924942,4.909240,-0.999271,-29.295285
2018-02-28,470.15,477.80,467.60,472.15,472.15,249134,-18.391727,-20.329021,1.937293,37.601510,...,17.293895,-17.668362,474.814286,-6.634368,Trend.SIDE,1.667813,1.667813,3.387670,-1.736736,-24.240613
2018-03-01,472.80,476.00,460.10,462.20,462.20,237698,-18.731933,-20.009603,1.277670,34.822712,...,-18.788608,-17.347756,468.104396,-6.729896,Trend.SIDE,3.383489,3.383489,4.078630,-2.152748,-18.926177
2018-03-05,459.95,459.95,445.50,451.70,451.70,210000,-19.622614,-19.932205,0.309591,32.124713,...,-41.690835,-17.624536,459.007692,-7.105398,Trend.SIDE,2.917944,2.917944,2.827333,-2.324552,-13.424633
2018-03-06,455.95,459.20,440.75,443.65,443.65,248865,-20.738986,-20.093562,-0.645425,30.193275,...,-48.147338,-18.255659,451.296703,-7.745893,Trend.SIDE,3.635162,3.635162,4.277664,-1.814493,-7.783926
